# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [8]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [9]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index2word

['to',
 'you',
 'the',
 'and',
 'is',
 'in',
 'my',
 'me',
 'it',
 'for',
 'your',
 'of',
 'call',
 'have',
 'that',
 'on',
 'now',
 'are',
 'so',
 'can',
 'but',
 'or',
 'not',
 'we',
 'get',
 'do',
 'at',
 'will',
 'if',
 'be',
 'ur',
 'no',
 'with',
 'just',
 'this',
 'up',
 'when',
 'how',
 'lt',
 'gt',
 'free',
 'ok',
 'out',
 'from',
 'what',
 'go',
 'll',
 'know',
 'all',
 'then',
 'day',
 'am',
 'good',
 'like',
 'there',
 'got',
 'he',
 'only',
 'was',
 'come',
 'time',
 'its',
 'send',
 'love',
 'text',
 'want',
 'by',
 'one',
 'txt',
 'as',
 'she',
 'need',
 'going',
 'stop',
 'still',
 'see',
 'today',
 'home',
 'don',
 'da',
 'back',
 'about',
 'lor',
 'take',
 'reply',
 'mobile',
 'sorry',
 'her',
 'our',
 'please',
 'any',
 'new',
 'dont',
 'tell',
 'phone',
 'been',
 'pls',
 'here',
 'later',
 'hi',
 'did',
 'they',
 'dear',
 'night',
 'an',
 'some',
 'who',
 'ì_',
 'hope',
 'think',
 'much',
 'claim',
 'great',
 'oh',
 'week',
 'happy',
 'hey',
 'more',
 'where',
 'msg

In [10]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index2word])
                     for ls in X_test])

/home/cherry/documents/projects/learnDataScience/machineLearning/Adv_NLP_Python_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Why is the length of the sentence different than the length of the sentence vector?
for i,v in enumerate(w2v_vect):
    print(len(X_test.iloc[i]),len(v))

17 17
4 4
49 37
4 4
24 20
15 13
7 7
8 8
10 10
13 12
6 6
23 23
49 41
17 7
5 5
20 20
11 11
6 6
19 16
17 17
8 8
7 7
22 19
6 6
25 20
8 7
12 12
16 16
3 3
2 2
13 12
24 23
5 5
21 20
8 7
9 8
20 15
9 6
6 6
10 10
13 13
5 5
5 5
27 27
8 8
17 17
29 29
16 15
29 27
9 9
4 4
7 6
5 5
26 20
14 14
9 8
16 12
1 1
10 10
10 9
9 8
12 11
8 5
6 6
9 8
7 7
5 5
26 25
25 24
20 17
10 10
1 1
6 6
21 21
11 10
6 6
21 21
22 19
6 5
10 10
24 23
17 14
9 8
9 9
9 8
9 8
6 6
30 30
4 4
13 11
17 16
29 28
6 6
17 16
6 6
7 5
6 5
7 7
7 6
17 17
13 11
15 14
5 4
27 25
18 17
14 13
11 10
8 8
13 12
9 9
10 10
3 3
12 11
5 4
6 6
5 3
12 12
27 25
5 1
7 4
9 9
9 8
4 4
29 28
9 9
16 14
12 12
17 13
11 11
9 9
12 12
18 17
11 11
9 9
5 5
5 2
12 9
9 8
35 34
8 8
10 9
20 18
12 12
21 17
27 23
5 5
12 10
13 13
19 19
14 13
16 13
7 5
11 11
8 5
60 56
7 5
25 24
7 7
24 23
25 23
10 9
11 11
1 0
16 15
26 25
27 25
29 28
27 27
21 18
10 10
21 18
8 8
23 22
5 3
8 8
26 26
14 13
19 19
11 11
5 5
18 18
22 17
14 13
5 5
21 20
10 10
16 16
22 21
6 5
6 6
14 13
16 15
21 20
9 8
9 9
4

In [16]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vect:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [17]:
# Are our sentence vector lengths consistent?
for i,v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]),len(v))

17 100
4 100
49 100
4 100
24 100
15 100
7 100
8 100
10 100
13 100
6 100
23 100
49 100
17 100
5 100
20 100
11 100
6 100
19 100
17 100
8 100
7 100
22 100
6 100
25 100
8 100
12 100
16 100
3 100
2 100
13 100
24 100
5 100
21 100
8 100
9 100
20 100
9 100
6 100
10 100
13 100
5 100
5 100
27 100
8 100
17 100
29 100
16 100
29 100
9 100
4 100
7 100
5 100
26 100
14 100
9 100
16 100
1 100
10 100
10 100
9 100
12 100
8 100
6 100
9 100
7 100
5 100
26 100
25 100
20 100
10 100
1 100
6 100
21 100
11 100
6 100
21 100
22 100
6 100
10 100
24 100
17 100
9 100
9 100
9 100
9 100
6 100
30 100
4 100
13 100
17 100
29 100
6 100
17 100
6 100
7 100
6 100
7 100
7 100
17 100
13 100
15 100
5 100
27 100
18 100
14 100
11 100
8 100
13 100
9 100
10 100
3 100
12 100
5 100
6 100
5 100
12 100
27 100
5 100
7 100
9 100
9 100
4 100
29 100
9 100
16 100
12 100
17 100
11 100
9 100
12 100
18 100
11 100
9 100
5 100
5 100
12 100
9 100
35 100
8 100
10 100
20 100
12 100
21 100
27 100
5 100
12 100
13 100
19 100
14 100
16 100
7 100
11 100